In [1]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import scipy
import cv2
import os
from tqdm import tqdm
import numpy as np
from skimage.segmentation import find_boundaries
from skimage.morphology import thin

# 1 -> Frontal wall
# 2 -> Left wall
# 3 -> Right wall
# 4 -> Floor 
# 5 -> Ceiling 

img_shape = (256, 256)

In [20]:
data_dir = "LSUN2016_surface_relabel/surface_relabel/train"

files = os.listdir(data_dir)

jpg_files = [f for f in files if f.endswith('.jpg')]
mat_files = [f for f in files if f.endswith('.mat')]

jpg_dict = {os.path.splitext(f)[0]: os.path.join(data_dir, f) for f in jpg_files}
mat_dict = {os.path.splitext(f)[0]: os.path.join(data_dir, f) for f in mat_files}

common_names = set(jpg_dict.keys()) & set(mat_dict.keys())

# imgs = np.empty((4000, *img_shape, 3), dtype=np.uint8)
# lbs = np.empty((4000, *img_shape), dtype=np.uint8)

# i = 0
# for name in tqdm(common_names):
#     img_path = jpg_dict[name]
#     mat_path = mat_dict[name]

#     img = cv2.imread(img_path)
#     lb = scipy.io.loadmat(mat_path)['layout']
    

#     imgs[i] = cv2.resize(img, img_shape)
#     lbs[i] = cv2.resize(lb, img_shape)
#     i += 1

In [3]:
def show_ds(img, lbl):
    n = len(img)
    fig, ax = plt.subplots(n, 2, figsize=(6, 8), squeeze=False)
    for i, (im, l) in enumerate(zip(img, lbl)):
        ax[i, 0].imshow(im)
        ax[i, 0].set_axis_off()
        ax[i, 1].imshow(l)
        ax[i, 1].set_axis_off()
    fig.tight_layout(pad=1)

In [4]:
# ids = 205
# show_ds(imgs[ids:ids + 3], lbs[ids:ids + 3])

NameError: name 'imgs' is not defined

In [21]:
files = os.listdir(data_dir)
paths = [os.path.splitext(f)[0] for f in files if f.endswith('.jpg')]

os.makedirs('./Dataset_floor', exist_ok=True)
for name in tqdm(paths[4:]):
    img = cv2.imread(f'{os.path.join(data_dir, name)}.jpg')
    lb = scipy.io.loadmat(f'{os.path.join(data_dir, name)}.mat')['layout']
    lb = np.where(lb == 4, 255, 0).astype(np.uint8)

    cv2.imwrite(f'./Dataset_floor/{name}.jpg', img)
    cv2.imwrite(f'./Dataset_floor/{name}.png', lb)

100%|█████████████████████████████████████████████████████| 4390/4390 [01:35<00:00, 46.16it/s]


In [55]:
l = lb.copy()

coarse_boundary = find_boundaries(l, mode='thick', connectivity=2).astype(np.uint8)
skeleton = thin(coarse_boundary).astype(np.uint8)
skeleton[90:100, 85:100]

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [57]:
kernel = np.array([[1, 1, 1],
                   [1, 0, 1],
                   [1, 1, 1]], dtype=np.uint8)

neighbor_count = convolve(skeleton, kernel, mode='constant', cval=0)
neighbor_count[90:100, 85:100]

In [64]:
from skimage import measure
r = np.where(skeleton & (neighbor_count == 3), 1, 0)

regions = measure.regionprops(r)

# r[90:100, 85:100]
for region in regions:
    print("Центр перекрёстка:", region.centroid)
    print("Округлённый центр:", tuple(np.round(region.centroid).astype(int)))

Центр перекрёстка: (np.float64(149.0), np.float64(92.33333333333333))
Округлённый центр: (np.int64(149), np.int64(92))


In [65]:
r[145:155, 88:95]

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])